# Vertex pipelines

**Learning Objectives:**

Use components from `google_cloud_pipeline_components` to create a Vertex Pipeline which will
  1. train a custom model on Vertex AI
  1. create an endpoint to host the model 
  1. upload the trained model, and 
  1. deploy the uploaded model to the endpoint for serving

## Overview

This notebook shows how to use the components defined in [`google_cloud_pipeline_components`](https://github.com/kubeflow/pipelines/tree/master/components/google-cloud) in conjunction with an experimental `run_as_aiplatform_custom_job` method, to build a [Vertex Pipelines](https://cloud.google.com/vertex-ai/docs/pipelines) workflow that trains a [custom model](https://cloud.google.com/vertex-ai/docs/training/containers-overview), uploads the model, creates an endpoint, and deploys the model to the endpoint. 

We'll use the `kfp.v2.google.experimental.run_as_aiplatform_custom_job` method to train a custom model.

The google cloud pipeline components are [documented here](https://google-cloud-pipeline-components.readthedocs.io/en/google-cloud-pipeline-components-0.1.2/). From this [github page](...) you can also find other examples in how to build a Vertex pipeline with AutoML [here](https://github.com/GoogleCloudPlatform/ai-platform-samples/tree/master/ai-platform-unified/notebooks/official/pipelines). You can see other available methods from the [Vertex AI SDK](https://googleapis.dev/python/aiplatform/latest/aiplatform.html).

### Set up your local development environment and install necessary packages



In [1]:
!pip3 install --user google-cloud-pipeline-components==0.1.1 --upgrade

### Restart the kernel

After you install the additional packages, you need to restart the notebook kernel so it can find the packages.

### Import libraries and define constants

In [2]:
import os
from datetime import datetime

import kfp
from google.cloud import aiplatform
from google_cloud_pipeline_components import aiplatform as gcc_aip
from kfp.v2 import compiler
from kfp.v2.dsl import component
from kfp.v2.google import experimental

Check the versions of the packages you installed.  The KFP SDK version should be >=1.6.

In [3]:
print(f"KFP SDK version: {kfp.__version__}")

KFP SDK version: 1.8.10


#### Set your environment variables
Next, we'll set up our project variables, like GCP project ID, the bucket and region. Also, to avoid name collisions between resources created, we'll create a timestamp and append it onto the name of resources we create in this lab.

In [4]:
# Change below if necessary
PROJECT = !gcloud config get-value project  # noqa: E999
PROJECT = PROJECT[0]
BUCKET = PROJECT
REGION = "us-central1"

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

PIPELINE_ROOT = f"gs://{BUCKET}/pipeline_root"

In [5]:
print(PIPELINE_ROOT)

gs://qwiklabs-gcp-04-853e5675f5e8/pipeline_root


We'll save pipeline artifacts in a directory called `pipeline_root` within our bucket. Validate access to your Cloud Storage bucket by examining its contents. It should be empty at this stage. 

In [6]:
!gsutil ls -la gs://{BUCKET}/pipeline_root

         0  2022-04-23T12:24:54Z  gs://qwiklabs-gcp-04-853e5675f5e8/pipeline_root/#1650716694684977  metageneration=1
                                 gs://qwiklabs-gcp-04-853e5675f5e8/pipeline_root/1076138843678/
TOTAL: 1 objects, 0 bytes (0 B)


### Give your default service account storage bucket access
This pipeline will read `.csv` files from Cloud storage for training and will write model checkpoints and artifacts to a specified bucket. So, we need to give our default service account `storage.objectAdmin` access. You can do this by running the command below in Cloud Shell:

```bash
PROJECT=$(gcloud config get-value project)
PROJECT_NUMBER=$(gcloud projects list --filter="name=$PROJECT" --format="value(PROJECT_NUMBER)")
gcloud projects add-iam-policy-binding $PROJECT \
    --member="serviceAccount:$PROJECT_NUMBER-compute@developer.gserviceaccount.com" \
    --role="roles/storage.objectAdmin"
```

Note, it may take some time for the permissions to propogate to the service account. You can confirm the status from the [IAM page here](https://console.cloud.google.com/iam-admin/iam). 

## Define a pipeline that uses the components


We'll start by defining a component with which the custom training job is run.  For this example, this component doesn't do anything (but run a print statement).

In [7]:
@component
def training_op(input1: str):
    print(f"VertexAI pipeline: {input1}")

Now, you define the pipeline.  

The `experimental.run_as_aiplatform_custom_job` method takes as args the component defined above, and the list of `worker_pool_specs`— in this case  one— with which the custom training job is configured. 
See [full function code here](https://github.com/kubeflow/pipelines/blob/master/sdk/python/kfp/v2/google/experimental/custom_job.py)

Then, [`google_cloud_pipeline_components`](https://github.com/kubeflow/pipelines/tree/master/components/google-cloud) components are used to define the rest of the pipeline: upload the model, create an endpoint, and deploy the model to the endpoint. (While not shown in this example, the model deploy will create an endpoint if one is not provided). 

Note that the code we're using the exact same code that we developed in the previous lab [`1_training_at_scale_vertex.ipynb`](1_training_at_scale_vertex.ipynb). In fact, we are pulling the same python package executor image URI that we pushed to Cloud storage in that lab. Note that we also include the `SERVING_CONTAINER_IMAGE_URI` since we'll need to specify that when uploading and deploying our model.

In [8]:
# Output directory and job_name
OUTDIR = f"gs://{BUCKET}/taxifare/trained_model_{TIMESTAMP}"
MODEL_DISPLAY_NAME = f"taxifare_{TIMESTAMP}"

PYTHON_PACKAGE_URIS = f"gs://{BUCKET}/taxifare/taxifare_trainer-0.1.tar.gz"
MACHINE_TYPE = "n1-standard-16"
REPLICA_COUNT = 1
PYTHON_PACKAGE_EXECUTOR_IMAGE_URI = (
    "us-docker.pkg.dev/vertex-ai/training/tf-cpu.2-3:latest"
)
SERVING_CONTAINER_IMAGE_URI = (
    "us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-3:latest"
)
PYTHON_MODULE = "trainer.task"

# Model and training hyperparameters
BATCH_SIZE = 500
NUM_EXAMPLES_TO_TRAIN_ON = 10000
NUM_EVALS = 1000
NBUCKETS = 10
LR = 0.001
NNSIZE = "32 8"

# GCS paths
GCS_PROJECT_PATH = f"gs://{BUCKET}/taxifare"
DATA_PATH = f"{GCS_PROJECT_PATH}/data"
TRAIN_DATA_PATH = f"{DATA_PATH}/taxi-train*"
EVAL_DATA_PATH = f"{DATA_PATH}/taxi-valid*"

### Lab Task #1. 

In the cell below we define the pipeline for training and deploying our taxifare model. Fill in the code to accomplish four things:
1. define the approrpriate `worker_pool_spec` for the training job
1. use `ModelUploadOp` to upload the model artifacts after training to create the model in Vertex AI
1. create an endpoing using `EndpointCreateOp`
1. finally, deploy the model you uploaded to the endpoint you created in the steps above.

In [9]:
@kfp.dsl.pipeline(name="taxifare--train-upload-endpoint-deploy")
def pipeline(
    project: str = PROJECT,
    model_display_name: str = MODEL_DISPLAY_NAME,
):
    train_task = training_op("taxifare training pipeline")
    experimental.run_as_aiplatform_custom_job(
        train_task,
        display_name=f"pipelines-train-{TIMESTAMP}",
        worker_pool_specs= [
        # TODO:
            {
                "pythonPackageSpec": {
                    "executor_image_uri": PYTHON_PACKAGE_EXECUTOR_IMAGE_URI,
                    "package_uris": [PYTHON_PACKAGE_URIS],
                    "python_module": PYTHON_MODULE,
                    "args": [
                        f"--eval_data_path={EVAL_DATA_PATH}",
                        f"--output_dir={OUTDIR}",
                        f"--train_data_path={TRAIN_DATA_PATH}",
                        f"--batch_size={BATCH_SIZE}",
                        f"--num_examples_to_train_on={NUM_EXAMPLES_TO_TRAIN_ON}",  # noqa: E501
                        f"--num_evals={NUM_EVALS}",
                        f"--nbuckets={NBUCKETS}",
                        f"--lr={LR}",
                        f"--nnsize={NNSIZE}",
                    ],
                },
                "replica_count": f"{REPLICA_COUNT}",
                "machineSpec": {
                    "machineType": f"{MACHINE_TYPE}",
                }
            }
        ] 
    )

    model_upload_op = gcc_aip.ModelUploadOp(
        # TODO:
        project=f"{PROJECT}",
        display_name=f"pipelines-ModelUpload-{TIMESTAMP}",
        artifact_uri=f"{OUTDIR}/savedmodel",
        serving_container_image_uri=f"{SERVING_CONTAINER_IMAGE_URI}",
        serving_container_environment_variables={"NOT_USED": "NO_VALUE"}
    )
    model_upload_op.after(train_task)

    endpoint_create_op = gcc_aip.EndpointCreateOp(
        # TODO:
        project=f"{PROJECT}",
        display_name=f"pipelines-EndpointCreate-{TIMESTAMP}",
    )
    model_deploy_op = gcc_aip.ModelDeployOp(
        # TODO:
        project=f"{PROJECT}",
        endpoint=endpoint_create_op.outputs["endpoint"],
        model=model_upload_op.outputs["model"],
        deployed_model_display_name=f"{MODEL_DISPLAY_NAME}",
        machine_type=f"{MACHINE_TYPE}"
    )

## Compile and run the pipeline

Now, you're ready to compile the pipeline:

In [10]:
if not os.path.isdir("vertex_pipelines"):
    os.mkdir("vertex_pipelines")

pipeline_json_fpath = "./vertex_pipelines/train_upload_endpoint_deploy.json"
compiler.Compiler().compile(
    pipeline_func=pipeline,
    package_path=pipeline_json_fpath,
)

/opt/conda/lib/python3.7/site-packages/kfp/v2/compiler/compiler.py:1266: FutureWarning: APIs imported from the v1 namespace (e.g. kfp.dsl, kfp.components, etc) will not be supported by the v2 compiler since v2.0.0
  category=FutureWarning,
/opt/conda/lib/python3.7/site-packages/kfp/v2/google/experimental/custom_job.py:78: FutureWarning: This function will be deprecated in v2.0.0
  'This function will be deprecated in v2.0.0', category=FutureWarning,


The pipeline compilation generates the `train_upload_endpoint_deploy.json` job spec file.

Next, instantiate the pipeline job object:

### Lab Task #2.

Complete the code in the cell below to fill in the missing arguments.


In [11]:
pipeline_json_fpath

'./vertex_pipelines/train_upload_endpoint_deploy.json'

In [12]:
# TODO:
# The path of PipelineJob or PipelineSpec JSON file
pipeline_job = aiplatform.pipeline_jobs.PipelineJob(
    display_name= "taxifare_pipeline",
    template_path= pipeline_json_fpath,  
    pipeline_root= "vertex_pipelines",
    project=PROJECT,
    location=REGION,
)

In [13]:
pipeline_job = aiplatform.pipeline_jobs.PipelineJob(
    display_name="taxifare_pipeline",
    template_path="./vertex_pipelines/train_upload_endpoint_deploy.json",
    pipeline_root=f"{PIPELINE_ROOT}",
    project=PROJECT,
    location=REGION,
)

Then, you run the defined pipeline like this: 

In [14]:
pipeline_job.run()

INFO:google.cloud.aiplatform.pipeline_jobs:Creating PipelineJob
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob created. Resource name: projects/1076138843678/locations/us-central1/pipelineJobs/taxifare--train-upload-endpoint-deploy-20220423124716
INFO:google.cloud.aiplatform.pipeline_jobs:To use this PipelineJob in another session:
INFO:google.cloud.aiplatform.pipeline_jobs:pipeline_job = aiplatform.PipelineJob.get('projects/1076138843678/locations/us-central1/pipelineJobs/taxifare--train-upload-endpoint-deploy-20220423124716')
INFO:google.cloud.aiplatform.pipeline_jobs:View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/taxifare--train-upload-endpoint-deploy-20220423124716?project=1076138843678
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob projects/1076138843678/locations/us-central1/pipelineJobs/taxifare--train-upload-endpoint-deploy-20220423124716 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aip

Click on the generated link above starting with `https://console.cloud.google.com/vertex-ai/locations/[location]/pipelines/runs/` to see your run in the Cloud Console.  It should look something like this:

<img src='../assets/taxifare_vertex_pipeline.png' width='80%'>

Copyright 2021 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License